# Python for Biostatistics
Welcome to this interactive tutorial where we'll explore the powerful capabilities of Python applied to the field of biostatistics. Biostatistics is a discipline that utilizes statistical techniques to help answer questions in the biological, medical, and health sciences. Whether you're a student, a medical researcher, or a data science enthusiast, understanding biostatistics is crucial in the analysis of your data and the interpretation of your results.

# Understanding Population and Sample Estimated Parameters

# Task 1: Generating Data

Create a synthetic dataset that represents our "population" data. We'll assume this data follows a normal distribution, often used in biological statistics for characteristics such as height, blood pressure, or reaction times.


In [1]:
import numpy as np

# Set a seed for reproducibility
np.random.seed(0)

# Generate population data
population_size = 100000
mean = 65  # hypothetical average value, e.g., average height
std_dev = 5  # standard deviation, representing the spread of the data
population_data = np.random.normal(mean, std_dev, population_size)

In [2]:
# Define sample size
sample_size = 100

# Randomly select data points from the population to be in the sample
sample_data = np.random.choice(population_data, sample_size, replace=False)

# Task 2: Calculating Sample Statistics

Now, calculate the basic statistics of your sample. These are the "estimates" of the population parameters.

In [3]:
sample_mean = np.mean(sample_data)
sample_std_dev = np.std(sample_data, ddof=1)  # Using ddof=1 for an unbiased estimate
sample_variance = np.var(sample_data, ddof=1)

print(f"Sample Mean: {sample_mean}")
print(f"Sample Standard Deviation: {sample_std_dev}")
print(f"Sample Variance: {sample_variance}")

Sample Mean: 65.19786447256085
Sample Standard Deviation: 5.062741891321793
Sample Variance: 25.631355458144565


# Task 3: Exploring Group Differences with t-tests and ANOVA
When dealing with continuous data, we often want to compare means across multiple groups. For instance, researchers might want to test if there is a significant difference in blood pressure between different patient groups treated with different medications.

*   A t-test is used when comparing the means between just two groups.
*   ANOVA (Analysis of Variance) is used when comparing three or more groups. It determines if there are any statistically significant differences between the means of three or more independent (unrelated) groups.


Before you start, let's assume you have data on the systolic blood pressure of patients administered three different types of medication.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats

# Assuming you've loaded your dataset as a CSV file
# data = pd.read_csv('your_data_file.csv')

# For demonstration, we're creating a synthetic dataset
np.random.seed(0)  # for reproducibility

data = pd.DataFrame({
    'blood_pressure': np.concatenate([np.random.normal(120, 5, 100),  # Medication A
                                      np.random.normal(115, 5, 100),  # Medication B
                                      np.random.normal(125, 8, 100)]), # Medication C
    'medication': np.concatenate([['A'] * 100, ['B'] * 100, ['C'] * 100])
})

data.head()  # preview the dataset

,blood_pressure,medication
0,128.820262,A
1,122.000786,A
2,124.893690,A
3,131.204466,A
4,129.337790,A


Say we want to compare the effectiveness of medication 'A' versus medication 'B' on systolic blood pressure.

In [5]:
# Separating the blood pressure readings based on the type of medication
group_A = data['blood_pressure'][data['medication'] == 'A']
group_B = data['blood_pressure'][data['medication'] == 'B']

# Perform a t-test
t_stat, p_val = stats.ttest_ind(group_A, group_B)

print(f"T-statistic: {t_stat}")
print(f"P-value: {p_val}")

T-statistic: 6.735511068041367
P-value: 1.732740880380291e-10


Now, we want to compare the effects of medications A, B, and C together.

In [6]:
group_C = data['blood_pressure'][data['medication'] == 'C']

# Perform one-way ANOVA
f_stat, p_val = stats.f_oneway(group_A, group_B, group_C)

print(f"F-statistic: {f_stat}")
print(f"P-value: {p_val}")

F-statistic: 56.109937536611085
P-value: 2.131866637422407e-21


# Task 4: Applying Fisher's Exact Test in Contingency Table Analysis
Suppose you're investigating the effect of a new drug on the incidence of a specific medical condition, and your data is categorical (e.g., "Condition Improved" or "No Improvement"). When sample sizes are small, traditional chi-square tests for analyzing contingency tables might not be appropriate due to the increased likelihood of Type I errors (false positives). In such cases, Fisher's exact test is preferable because it does not assume a "chi-square" distribution and is not affected by small expected frequencies.

Assume you conducted a clinical trial with a small group of patients to test a new drug. You recorded whether or not patients' conditions improved after treatment. The data looks like this:

Improved: 4 patients from the treatment group and 1 from the control group.
No Improvement: 1 patient from the treatment group and 4 from the control group.
Now, you'll use Fisher's exact test to determine if the improvement is associated with the treatment or occurs randomly.

In [7]:
import scipy.stats as stats

# Data preparation: Constructing a 2x2 contingency table
#                Treatment    Control
# Improved           4          1
# No Improvement     1          4

contingency_table = [[4, 1],
                     [1, 4]]

# Perform Fisher's exact test
odds_ratio, p_value = stats.fisher_exact(contingency_table)

print(f"Odds Ratio: {odds_ratio:.2f}")
print(f"P-value: {p_value:.4f}")

Odds Ratio: 16.00
P-value: 0.2063


# Task 5: Understanding and Applying False Discovery Rate (FDR) Correction
In research areas where multiple hypothesis testing is conducted, controlling the False Discovery Rate (FDR) is essential. FDR is the expected proportion of false positives among all significant results. By controlling FDR, we reduce the likelihood of incorrectly rejecting null hypotheses (Type I errors).

Imagine a scenario where you're analyzing gene expression data trying to identify genes that are differentially expressed between two conditions, say, normal tissue and cancerous tissue. Each hypothesis test corresponds to one gene, and you have expression data for thousands of genes.

For this exercise, we'll simulate p-values for multiple genes and then apply FDR correction.

Step 1: Simulating p-values

First, we'll simulate the scenario where we conduct 3000 tests, and 300 of these are true positives (there is a real effect), and the rest are null (no actual effect).

In [8]:
import numpy as np

np.random.seed(0)  # for reproducibility

# Assume we have 3000 tests
num_tests = 3000

# Simulate p-values: for simplicity, we'll use uniform distribution for null hypotheses
# and a different distribution for "true" hypotheses (to simulate some statistical effect)

# We have 2700 'null' p-values and 300 'true' p-values
p_values = np.concatenate([np.random.uniform(0, 1, 2700), np.random.uniform(0, 0.1, 300)])

# Shuffle the p-values
np.random.shuffle(p_values)

Step 2: Applying FDR correction

We will use the Benjamini-Hochberg procedure, one of the most common methods to control FDR.

In [9]:
from statsmodels.stats import multitest

# Perform FDR correction using Benjamini-Hochberg procedure
fdr_corrected_pvals = multitest.multipletests(p_values, method='fdr_bh')[1]

# Let's assume our significance threshold is 0.05
significant_indices = np.where(fdr_corrected_pvals < 0.05)[0]
num_significant = len(significant_indices)

print(f"Number of significant tests after FDR correction: {num_significant}")

Number of significant tests after FDR correction: 0
